In [1]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
import requests
import gmaps
import json
from config import g_key

In [2]:
# add in csv file from part I 
cities_2=pd.read_csv("../Weatherpy/unique_cities.csv")
cities_2=cities_2.dropna(how='any')
cities_2.head()

,Unnamed: 0,City,Cloudiness (%),Country,Date,Humidity (%),Lat,Lng,Max Temp (F),Wind Speed (mph)
0,0,Bengkulu,100.0,ID,2020-10-25 16:41:45,88.0,-3.80,102.27,75.218,1.42
2,2,Quang ngai,96.0,VN,2020-10-25 16:41:45,92.0,15.12,108.80,75.668,0.45
3,3,Rikitea,100.0,PF,2020-10-25 16:32:59,81.0,-23.12,-134.97,74.534,9.17
4,4,Saskylakh,100.0,RU,2020-10-25 16:41:45,93.0,71.92,114.08,8.294,5.03
5,5,Dikson,100.0,RU,2020-10-25 16:38:08,97.0,73.51,80.55,18.410,5.94


In [3]:
#create a heat map to demonstrate the humidity for each city 

gmaps.configure(api_key=g_key)

locations=cities_2[["Lat", "Lng"]]
humidity=cities_2["Humidity (%)"].astype(float)
humidity.max()

100.0

In [4]:
#plotting heatmap 

fig=gmaps.figure()

#create a heat layer
heat_layer=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#creating vacation spot limitations

cloud=cities_2[cities_2["Cloudiness (%)"]==0]
temp_above_70=cloud[cloud["Max Temp (F)"]>70]
temp_below_80=temp_above_70[temp_above_70["Max Temp (F)"] <80]
wind=temp_below_80[temp_below_80["Wind Speed (mph)"] <10]
vacay_spot=wind[wind["Humidity (%)"]<60]

perfect_vacay=vacay_spot.reset_index()
del perfect_vacay["index"]
perfect_vacay.head()

,Unnamed: 0,City,Cloudiness (%),Country,Date,Humidity (%),Lat,Lng,Max Temp (F),Wind Speed (mph)
0,62,Aswan,0.0,EG,2020-10-25 16:40:49,32.0,24.09,32.91,75.866,6.38
1,172,Umm lajj,0.0,SA,2020-10-25 16:42:09,53.0,25.02,37.27,79.538,1.91
2,240,Salalah,0.0,OM,2020-10-25 16:39:09,57.0,17.02,54.09,77.270,2.10
3,293,Erzin,0.0,TR,2020-10-25 16:42:26,53.0,36.96,36.20,76.262,4.02
4,500,Lalsot,0.0,IN,2020-10-25 16:42:56,36.0,26.57,76.33,75.470,1.68


In [6]:
vacay_loc=perfect_vacay[["Lat","Lng"]]
vacay_hum=perfect_vacay["Humidity (%)"].astype(float)

vacay_fig=gmaps.figure()

vacay_heat_layer=gmaps.heatmap_layer(vacay_loc, weights=vacay_hum, dissipating=False, max_intensity=59, point_radius=3)

vacay_fig.add_layer(vacay_heat_layer)

vacay_fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#find hotel within 5000 meters from coords
#plot the hotels on top of the humidity heat map, each pin containing Hotel Name, City and Country

hotels=[]

for city in range(len(perfect_vacay["City"])):
    
    lat=perfect_vacay.loc[city]["Lat"]
    lng=perfect_vacay.loc[city]["Lng"]
    
    city_coords=f"{lat},{lng}"
    
    params={
        "location":city_coords,
        "types":"lodging",
        "radius": 5000,
        "key":g_key
    }
    
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_request=requests.get(base_url,params=params)
    hotel_response=hotel_request.json()
    
    try:
        hotels.append(hotel_response["results"][1]["name"])
    except: 
        hotels.append("Nearest hotel not found")
        
perfect_vacay["Nearest Hotels"]=hotels
perfect_vacay

,Unnamed: 0,City,Cloudiness (%),Country,Date,Humidity (%),Lat,Lng,Max Temp (F),Wind Speed (mph),Nearest Hotels
0,62,Aswan,0.0,EG,2020-10-25 16:40:49,32.0,24.09,32.91,75.866,6.38,Nubian Beach Hostel
1,172,Umm lajj,0.0,SA,2020-10-25 16:42:09,53.0,25.02,37.27,79.538,1.91,عبدالله ظافر الثوعي
2,240,Salalah,0.0,OM,2020-10-25 16:39:09,57.0,17.02,54.09,77.270,2.10,City Hotel
3,293,Erzin,0.0,TR,2020-10-25 16:42:26,53.0,36.96,36.20,76.262,4.02,Erzin Belediyesi Şifalı Sular Moteli
4,500,Lalsot,0.0,IN,2020-10-25 16:42:56,36.0,26.57,76.33,75.470,1.68,YADAV HOUSE LALSOT
5,559,Hurghada,0.0,EG,2020-10-25 16:43:04,50.0,27.26,33.81,76.910,6.75,Bel Air Azur Resort
6,576,Acari,0.0,PE,2020-10-25 16:43:07,44.0,-15.43,-74.62,74.138,4.24,HOSPEDAJE XIMENA
7,589,Asyut,0.0,EG,2020-10-25 16:43:08,53.0,27.18,31.18,71.870,5.10,Assiut hotels Armed Forces


In [8]:
#Add the hotel marks to the heatmap 
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotels}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in perfect_vacay.iterrows()]
locations = perfect_vacay[["Lat", "Lng"]]



In [27]:
#adding marker layer ontop of heat map 

markers=gmaps.marker_layer(locations,info_box_content=[f"Nearest Hotel: {hotel}" for hotel in hotels])

vacay_fig.add_layer(markers)

vacay_fig

Figure(layout=FigureLayout(height='420px'))